In [166]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [ ]:
# Lendo o arquivo de dados processados

df = pd.read_csv('../data_processed/data_processed.csv', sep=',', encoding='latin1', low_memory = False)

In [ ]:
# Verificando as colunas existentes

df.columns

Index(['logradouro', 'municipio', 'latitude', 'longitude', 'tipo_registro',
       'turno', 'tp_sinistro_primario', 'qtd_gravidade_fatal',
       'qtd_gravidade_grave', 'qtd_gravidade_leve', 'qtd_caminhao',
       'qtd_motocicleta', 'qtd_automovel', 'qtd_pedestre', 'qtd_bicicleta',
       'qtd_onibus', 'qtd_veic_outros', 'qtd_veic_nao_disponivel',
       'concessionaria', 'numero_logradouro', 'tp_sinistro_atropelamento',
       'tp_sinistro_colisao_frontal', 'tp_sinistro_colisao_lateral',
       'tp_sinistro_colisao_transversal', 'tp_sinistro_colisao_outros',
       'tp_sinistro_choque', 'tp_sinistro_capotamento',
       'tp_sinistro_engavetamento', 'tp_sinistro_tombamento',
       'tp_sinistro_outros', 'data_hora', 'hora', 'mes', 'dia_semana',
       'indice_severidade', 'feriado', 'geometry', 'id', 'motorway', 'oneway',
       'lanes', 'maxspeed', 'bridge', 'dist_imprecisao'],
      dtype='object')

In [ ]:
# Definindo a variável alvo como 1 para os dados de acidentes reais

df['target'] = 1

In [ ]:
# Removendo colunas desnecessárias para a geração de dados

columns_drop = ['municipio', 'tipo_registro', 'turno', 'tp_sinistro_primario', 'qtd_gravidade_fatal', 
                'qtd_gravidade_grave', 'qtd_gravidade_leve', 'qtd_caminhao', 'qtd_motocicleta', 'qtd_automovel', 'qtd_pedestre', 
                'qtd_bicicleta', 'qtd_onibus', 'qtd_veic_outros', 'qtd_veic_nao_disponivel', 'numero_logradouro', 'tp_sinistro_atropelamento',
                'tp_sinistro_colisao_frontal', 'tp_sinistro_colisao_lateral', 'tp_sinistro_colisao_transversal', 'tp_sinistro_colisao_outros', 
                'tp_sinistro_choque', 'tp_sinistro_capotamento', 'tp_sinistro_engavetamento', 'tp_sinistro_tombamento', 'tp_sinistro_outros', 
                'data_hora', 'geometry', 'id', 'dist_imprecisao']

df = df.drop(axis = 1, columns = columns_drop)

In [ ]:
# Conferindo as colunas restantes

df.columns

Index(['logradouro', 'latitude', 'longitude', 'concessionaria', 'hora', 'mes',
       'dia_semana', 'indice_severidade', 'feriado', 'motorway', 'oneway',
       'lanes', 'maxspeed', 'bridge', 'target'],
      dtype='object')

In [ ]:
# Convertendo o dataframe para GeoDataFrame e ajustando projeção

geometry = gpd.points_from_xy(df['longitude'], df['latitude'])

gdf = gpd.GeoDataFrame(df, geometry=geometry)

gdf.set_crs(epsg=4326, inplace=True)

gdf = gdf.to_crs(epsg=31983)

In [ ]:
# Criando um buffer de 2km ao redor dos pontos de acidente

gdf['geometry'] = gdf.buffer(distance=2000)

In [ ]:
# Unificando os buffers para criar uma máscara de área de interesse

area_mask = gdf['geometry'].unary_union

area_mask = area_mask.simplify(tolerance=20)

C:\Users\Arthur Rios\AppData\Local\Temp\ipykernel_6048\4123187060.py:1: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  area_mask = gdf['geometry'].unary_union


In [ ]:
# Carregando a malha viária do arquivo parquet

columns_parquet = ['geometry', 'highway', 'oneway', 'lanes', 'maxspeed', 'bridge']

geodf = gpd.read_parquet('../data_processed/final_map_sp.parquet', columns=columns_parquet)

geodf = geodf.to_crs(epsg=31983)

In [ ]:
# Recortando a malha viária usando a máscara da área de interesse

geodf = gpd.clip(geodf, mask=area_mask)

In [ ]:
# Filtrando para manter apenas rodovias principais

mask = (geodf['highway'] == 'tertiary') | (geodf['highway'] == 'secondary') | (geodf['highway'] == 'primary')

geodf = geodf[~mask]

In [ ]:
# Calculando o comprimento de cada segmento de rodovia

geodf['geometry'].length

48629      372.725814
48628      370.977005
63361     2067.424263
48627     2063.065474
141890    1204.413273
             ...     
183296     140.665015
183298     167.433652
183297     141.558665
151695     433.161071
165947     123.835585
Length: 8491, dtype: float64

In [ ]:
# Definindo probabilidades de amostragem baseadas no comprimento

geometry_lenghts = geodf['geometry'].length

probability = geometry_lenghts/geometry_lenghts.sum()
probability = probability/probability.sum()

In [ ]:
# Realizando amostragem aleatória ponderada dos segmentos

index_list = np.random.choice(
    a=geodf.index.values,
    size=73000,
    replace=True,
    p=probability
)

In [ ]:
# Criando novo dataframe com os segmentos selecionados

newdf = geodf.loc[index_list]

In [ ]:
# Gerando pontos aleatórios ao longo dos segmentos selecionados

points = np.random.rand(73000)

newdf['geometry'] = newdf['geometry'].interpolate(points, normalized=True)

In [ ]:
# Definindo a variável alvo como 0 para os dados sintéticos

newdf['target'] = 0

In [ ]:
# Definindo índice de severidade zero para os não-acidentes

newdf['indice_severidade'] = 0

In [ ]:
# Definindo probabilidades para os dias da semana baseada no fluxo

days_probability = [0.14, 0.14, 0.14, 0.14, 0.17, 0.13, 0.14]

days_list = np.random.choice(
    a=[0, 1, 2, 3, 4, 5, 6],
    size=73000,
    replace=True,
    p=days_probability
)

In [ ]:
# Atribuindo dias da semana aleatórios baseados na probabilidade

newdf['dia_semana'] = days_list

In [ ]:
# Definindo probabilidades para os meses do ano baseada na fluxo

months_probability = np.array([0.100, 0.077, 0.077, 0.077, 0.077, 0.077, 0.100, 0.077, 0.077, 0.077, 0.077, 0.100])
months_probability = months_probability / months_probability.sum()

months_list = np.random.choice(
    a=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    size=73000,
    replace=True,
    p=months_probability
)

In [ ]:
# Atribuindo meses aleatórios

newdf['mes'] = months_list

In [ ]:
# Definindo probabilidade para feriados (com base na razão de números de feriados por dias do ano)

holiday_probability = [0.96, 0.04]

holiday_list = np.random.choice(
    a=[0, 1],
    size=73000,
    replace=True,
    p=holiday_probability
)

In [ ]:
# Atribuindo flag de feriado aleatoriamente

newdf['feriado'] = holiday_list

In [ ]:
# Definindo probabilidades para as horas do dia baseadas no fluxo

hours_probability = [0.01, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.06, 0.06, 0.06, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 
                     0.05, 0.06, 0.06, 0.06, 0.05, 0.05, 0.05, 0.01]

hours_list = np.random.choice(
    a=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
    size=73000,
    replace=True,
    p=hours_probability
)

In [ ]:
# Atribuindo horas aleatórias

newdf['hora'] = hours_list

In [ ]:
# Verificando a distribuição dos tipos de rodovia nos dados gerados

newdf['highway'].value_counts(dropna=False)

highway
motorway    62034
trunk       10966
Name: count, dtype: int64

In [ ]:
# Verificando a distribuição de mão única

newdf['oneway'].value_counts(dropna=False)

oneway
yes     70531
no       2000
None      469
Name: count, dtype: int64

In [ ]:
# Verificando a distribuição de faixas

newdf['lanes'].value_counts(dropna=False)

lanes
2       51890
3       12755
4        3504
None     2258
5        1428
1         610
6         180
8          84
7          80
10         78
9          34
18         25
14         25
11         18
12         13
21         10
15          4
20          2
16          1
13          1
Name: count, dtype: int64

In [ ]:
# Verificando a distribuição de limites de velocidade

newdf['maxspeed'].value_counts(dropna=False)

maxspeed
110     25011
120     14464
100     13375
80       6410
90       4813
60       2851
None     2471
50       1839
70       1079
40        673
30          8
20          2
45          2
35          2
Name: count, dtype: int64

In [ ]:
# Removendo dados gerados com infraestrutura incompleta

columns_dropna = ['oneway', 'lanes', 'maxspeed']

newdf = newdf.dropna(subset=columns_dropna)

In [ ]:
# Convertendo mão única para binário nos dados sintéticos

newdf['oneway'] = newdf['oneway'].replace('yes', 1)

newdf['oneway'] = newdf['oneway'].replace('no', 0)

C:\Users\Arthur Rios\AppData\Local\Temp\ipykernel_6048\2687004307.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  newdf['oneway'] = newdf['oneway'].replace('no', 0)


In [ ]:
# Convertendo colunas para inteiro

newdf['oneway'] = newdf['oneway'].astype('int')
newdf['lanes'] = newdf['lanes'].astype('int')
newdf['maxspeed'] = newdf['maxspeed'].astype('int')

In [ ]:
# Filtrando outliers de faixas

mask = newdf['lanes'] > 8

newdf = newdf[~mask]

In [ ]:
# Padronizando velocidades quebradas

to_replace = [35, 45]
value = [30, 40]

newdf['maxspeed'] = newdf['maxspeed'].replace(value=value, to_replace=to_replace)

In [ ]:
# Verificando distribuição de pontes

newdf['bridge'].value_counts(dropna=False)

bridge
None       66743
yes         1225
viaduct      557
Name: count, dtype: int64

In [ ]:
# Padronizando classificação de pontes e viadutos

newdf['bridge'] = newdf['bridge'].fillna(0)

newdf['bridge'] = newdf['bridge'].replace('yes', 1)

newdf['bridge'] = newdf['bridge'].replace('viaduct', 1)

C:\Users\Arthur Rios\AppData\Local\Temp\ipykernel_6048\3072185598.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  newdf['bridge'] = newdf['bridge'].replace('viaduct', 1)


In [ ]:
# Convertendo pontes para inteiro

newdf['bridge'] = newdf['bridge'].astype('int')

In [ ]:
# Convertendo tipo de rodovia para binário

newdf['highway'] = newdf['highway'].replace('motorway', 1)
newdf['highway'] = newdf['highway'].replace('trunk', 0)

C:\Users\Arthur Rios\AppData\Local\Temp\ipykernel_6048\4084116460.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  newdf['highway'] = newdf['highway'].replace('trunk', 0)


In [ ]:
# Renomeando coluna para motorway

newdf.rename(columns={'highway': 'motorway'}, inplace=True)

In [ ]:
#highways = newdf['highway'].unique()

#for highway in highways:

#    counts = newdf.loc[newdf['highway'] == highway, 'maxspeed'].value_counts(normalize=True)
#    maxspeeds = counts.index
#    prob = counts.values

#    maxspeed_list = np.random.choice(
#    a=maxspeeds,
#    size=len(newdf[(newdf['highway'] == highway) & (newdf['maxspeed'].isnull())]),
#    replace=True,
#    p=prob
#    )
#
#    newdf.loc[(newdf['highway'] == highway) & (newdf['maxspeed'].isnull()), 'maxspeed'] = maxspeed_list

In [ ]:
# Convertendo dataframe original para GeoDataFrame para o spatial join reverso

gdf = gpd.GeoDataFrame(
    df, 
    geometry=gpd.points_from_xy(df['longitude'], df['latitude']),
    crs="EPSG:4326"
)

In [ ]:
# Realizando spatial join para recuperar dados (concessionária/logradouro) dos pontos gerados

gdf = gdf.to_crs(newdf.crs)

datagendf = gpd.sjoin_nearest(
    newdf,
    gdf[['geometry', 'logradouro', 'concessionaria']], 
    how='left',
    distance_col='dist_imprecision',
)

datagendf = datagendf.drop_duplicates(subset=['geometry'])

In [ ]:
# Analisando a precisão do join reverso

datagendf['dist_imprecision'].describe()

count    68525.000000
mean       371.576135
std        467.226136
min          0.078350
25%         63.127453
50%        172.234736
75%        452.550016
max       2010.226541
Name: dist_imprecision, dtype: float64

In [ ]:
# Verificando distribuição por rodovia nos dados sintéticos

datagendf['logradouro'].value_counts()

logradouro
SP 330    15900
SP 270    12820
SP 310    11000
SP 280    10526
SP 070     7566
SP 348     6397
SP 150     3771
SP 123      545
Name: count, dtype: int64

In [ ]:
# Verificando distribuição por concessionária nos dados sintéticos

datagendf['concessionaria'].value_counts()

concessionaria
DER                       20782
AUTOBAN                    9472
ECOPISTAS                  7566
NAO DISPONIVEL             7462
ECONOROESTE                5543
CART                       4599
SPVIAS                     2572
EIXOSP - PIPA              1864
ENTREVIAS                  1833
ECOVIAS                    1514
VIAOESTE                   1244
INTERVIAS                  1102
ROTA SOROCABANA            1012
ECOVIAS RAPOSO-CASTELO      962
COLINAS                     595
VIAPAULISTA                 403
Name: count, dtype: int64

In [ ]:
# Conferindo colunas do dataset original

df.columns

Index(['logradouro', 'latitude', 'longitude', 'concessionaria', 'hora', 'mes',
       'dia_semana', 'indice_severidade', 'feriado', 'motorway', 'oneway',
       'lanes', 'maxspeed', 'bridge', 'target'],
      dtype='object')

In [ ]:
# Conferindo colunas do dataset sintético

datagendf.columns

Index(['geometry', 'motorway', 'oneway', 'lanes', 'maxspeed', 'bridge',
       'target', 'indice_severidade', 'dia_semana', 'mes', 'feriado', 'hora',
       'index_right', 'logradouro', 'concessionaria', 'dist_imprecision'],
      dtype='object')

In [ ]:
# Removendo colunas auxiliares antes da união

df_drop_col = ['latitude', 'longitude']
datagendf_drop_col = ['index_right', 'geometry', 'dist_imprecision']

df = df.drop(df_drop_col, axis=1)
datagendf = datagendf.drop(datagendf_drop_col, axis=1)

In [ ]:
# Concatenando dados reais e sintéticos

df = pd.concat([df, datagendf])

In [ ]:
# Aplicando One-Hot Encoding nas variáveis categóricas

columns_encode = ['logradouro', 'concessionaria']

df = pd.get_dummies(
    data = df,
    columns = columns_encode,
    dtype=int
)

In [ ]:
# Verificando estrutura final das colunas

df.columns

Index(['hora', 'mes', 'dia_semana', 'indice_severidade', 'feriado', 'motorway',
       'oneway', 'lanes', 'maxspeed', 'bridge', 'target', 'logradouro_SP 070',
       'logradouro_SP 123', 'logradouro_SP 150', 'logradouro_SP 270',
       'logradouro_SP 280', 'logradouro_SP 310', 'logradouro_SP 330',
       'logradouro_SP 348', 'concessionaria_AUTOBAN', 'concessionaria_CART',
       'concessionaria_COLINAS', 'concessionaria_DER',
       'concessionaria_ECONOROESTE', 'concessionaria_ECOPISTAS',
       'concessionaria_ECOVIAS', 'concessionaria_ECOVIAS RAPOSO-CASTELO',
       'concessionaria_EIXOSP - PIPA', 'concessionaria_ENTREVIAS',
       'concessionaria_INTERVIAS', 'concessionaria_NAO DISPONIVEL',
       'concessionaria_ROTA SOROCABANA', 'concessionaria_SPVIAS',
       'concessionaria_VIAOESTE', 'concessionaria_VIAPAULISTA'],
      dtype='object')

In [ ]:
# Verificando dimensão final do dataset pronto para modelagem

df.shape

(91099, 35)

In [ ]:
# Salvando o arquivo final processado

df.to_csv('data_machine_learning.csv', index=False)